In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Creating a fact transaction table  

In [0]:
%sql
-- creating a fact_transactions external table with Azure Data Lake Gen2 as the data source 
create table if not exists gold.fact_transactions(
  TransactionID string,
  SRC_TransactionID string,
  FK_ClaimID string,
  FK_EncounterID string,
  FK_PatientID string,
  FK_ProviderID string,
  FK_DeptID string,
  VisitType string,
  ServiceDate date,
  PaidDate date,
  ChargeAmount double,
  PaidAmount double,
  AmountType string,
  datasource string,
  refreshed_at timestamp
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/fact_transactions"

In [0]:
%sql
-- resetting the table

truncate table gold.fact_transactions


In [0]:
%sql
-- inserting into the fact table

insert into gold.fact_transactions
select 
  t.TransactionID, 
  t.SRC_TransactionID,
  t.ClaimID as FK_ClaimID,
  t.EncounterID as FK_EncounterID,
  concat(t.PatientID,'-',t.datasource ) as FK_Patient_ID,
  case when t.datasource='hos-a' then concat('H1-',t.providerID) else concat('H2-',t.providerID ) end as FK_Provider_ID, 
  concat(t.DeptID,'-',t.datasource ) as FK_Dept_ID, 
  t.VisitType,
  t.ServiceDate, 
  t.PaidDate,
  t.Amount, 
  t.PaidAmount, 
  t.AmountType,
  t.datasource,
  current_timestamp()
  from silver.transactions t 
  where t.is_current=true and t.is_quarantined=false


In [0]:
%sql
-- displaying some records
select * from gold.fact_transactions;

In [0]:
%sql
SHOW columns from gold.fact_transactions;